### Rozwiązywany jest opisany podczas zajęć problem optymalizacji zatrudnień pracowników w sklepie w ciągu tygodnia pod względem minimalizacji kosztów
każdy z pracowników posiada swoje stawki za godzinę, może być zatrudniony na 4 godziny dziennie oraz posiada określone dni w ktore może pracować  
ponadto każdy z pracowników może pracować maksymalnie 3 dni w tygodniu

![Zdj tabeli](./images/sklep.png)

#### problem został sprowadzony do problemu liniowego z  zastosowaniem zmiennych binarnych według nastepujacego schematu:

![Zdj tabeli binarnej](./images/sklep3.png)

dla każdej pary pracownik_dzien tygodnia została utworzona zmienna binarna reprezentujaca czy pracownik danego dnia pracuje czy nie  
w odniesieniu do tych zmiennych zostały utworzone ograniczenia w postaci przykładowo:  
suma dni w jakie pracownik pracuje (zmiennych go dotyczących przyjmujących wartość 1) musi być mniejsza od maksymalnej ilości dni jaką może przepracować w tygodniu

In [1]:
import pulp
print(pulp.__version__)

2.3.1


In [2]:
from pulp import *

In [74]:
dni_ludzie=[[0 for x in range(5)] for x in range(6)]
print(dni_ludzie)
#6 wierszy po 5 kolumn
# kolumny - dni, wiersze - ludzie

dni = ["pn", "wt", "sr", "czw", "pt"]
ludzie = ["Ania", "Stefan", "Hektor", "Olaf", "Lidia", "Piotr"]
i = 0
j = 0
for ludz in ludzie:
    for dzien in dni:
        dni_ludzie[ludzie.index(ludz)][dni.index(dzien)] = str(dzien) + str(ludz)

print(dni_ludzie)


#wymagania dni
dni_demand = [3,1,1,1,2]
stawki = [100, 50, 60, 40, 110, 70]

[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]
[['pnAnia', 'wtAnia', 'srAnia', 'czwAnia', 'ptAnia'], ['pnStefan', 'wtStefan', 'srStefan', 'czwStefan', 'ptStefan'], ['pnHektor', 'wtHektor', 'srHektor', 'czwHektor', 'ptHektor'], ['pnOlaf', 'wtOlaf', 'srOlaf', 'czwOlaf', 'ptOlaf'], ['pnLidia', 'wtLidia', 'srLidia', 'czwLidia', 'ptLidia'], ['pnPiotr', 'wtPiotr', 'srPiotr', 'czwPiotr', 'ptPiotr']]


### Zmienne zostaly zapisane w postaci listy list, co o rozmiarach odpowiadajacym 6 pracownikom i 5 dniom tygodnia

przygotowane listy sa wykorzystywane w problemie liniowym do utworzenia zmiennych liniowych
następnie opisane wcześniej ograniczenia są aplikowane dla odpowiednich zmiennych list (zawartych w liście)
w ten sposób rozpatrywane są możliwości tygodniowe każdej z osób
dodatowo niektóre zmienne muszą przyjąć wartość zero (kolejne ograniczenia), gdyż w te dni niektóre z osób nie są dostępne

In [98]:
prob = LpProblem("ProblemSklepu", LpMinimize)
#zmienne

#x = LpVariable.dict('lista', dni_ludzie, lowBound = 0, upBound = 1, cat = LpInteger)
#nope, nie dziala z lista, ale z tuplami tak? nie, też nie
#for x in range(6):
#    for y in range(5):
#        var[x][y] = LpVariable(dni_ludzie[x][y],0,3,LpInteger)
#var = [[LpVariable("x{}".format(i+j), 0, 3, LpInteger) for i in range(5)]for j in range(6)]


var = [[LpVariable("{}".format(dzien+ludz), 0, 1, LpInteger) for dzien in dni]for ludz in ludzie] #0,1 bo binarne
#print(var[0][0])

#funkcja zysku
prob+= 4*lpSum( [[stawki[ludzie.index(ludz)]*var[ludzie.index(ludz)][dni.index(dzien)] for ludz in ludzie] for dzien in dni] ), "Koszt"

#wymagania dla kazdego dnia
#poniewaz index znajduje pierwszy index pierwszego elementu o danej wartosci a nie bezwzgledny
i = 0
#jeszcze lepsze byloby enumerate!
for dzien_d in dni_demand:
    prob+= lpSum([(var[ludzie.index(ludz)][i]) for ludz in ludzie]) >= dzien_d
    i+=1
    print(i,":",dzien_d)

    
#mozliwosci Anii
for dnA in var[0]:
    prob += lpSum(var[0][var[0].index(dnA)]) <=5

#stefan moze tylko w pon
for i in range(1,5):
    var[1][i] == 0

#hekt
var[2][3] == 0
var[2][4] ==0

#olaf
#wlasciwie <=5 to nie ograniczenie

#lidia
var[4][1] == 0
var[4][2] == 0
var[4][3] == 0

#piotr
var[5][3] == 0
var[5][4] == 0

for i in range(6): #6 ludzi, juz mi sie nie chce list index
    for j in range(5):#5 dni
        prob += lpSum(var[i][j]) <=3
prob.solve()
#for dzien_d in dni_demand:
#    lpSum(var[dni_demand.index(dzien_d)])

#czy rozwiazalo
print("Status:", LpStatus[prob.status])
#dobrane wartosci zmiennych
for v in prob.variables():
    print(v.name, "=", v.varValue)
#wartosc zoptymalizowanej funkcji (zysk)
print("Calkowity koszt =", value(prob.objective))

1 : 3
2 : 1
3 : 1
4 : 1
5 : 2
Status: Optimal
czwAnia = 0.0
czwHektor = 0.0
czwLidia = 0.0
czwOlaf = 1.0
czwPiotr = 0.0
czwStefan = 0.0
pnAnia = 0.0
pnHektor = 1.0
pnLidia = 0.0
pnOlaf = 1.0
pnPiotr = 0.0
pnStefan = 1.0
ptAnia = 0.0
ptHektor = 0.0
ptLidia = 0.0
ptOlaf = 1.0
ptPiotr = 0.0
ptStefan = 1.0
srAnia = 0.0
srHektor = 0.0
srLidia = 0.0
srOlaf = 1.0
srPiotr = 0.0
srStefan = 0.0
wtAnia = 0.0
wtHektor = 0.0
wtLidia = 0.0
wtOlaf = 1.0
wtPiotr = 0.0
wtStefan = 0.0
Calkowity koszt = 1440.0


### WNIOSKI
Program przygotował listę wszystkich zmienych reprezentujących który z pracowników pracuje którego dnia, a także zwrócił koszt takiego podziału zatrudnienia, który uznał za optymalny